# [chatbot]
- 참조: https://python.langchain.com/v0.2/docs/tutorials/chatbot/ 	


## [Quickstart]


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.chat_history import(BaseChatMessageHistory, InMemoryChatMessageHistory)
from langchain_core.runnables.history import RunnableWithMessageHistory
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model='gpt-3.5-turbo')

answer = model.invoke([HumanMessage(content="Hi. I'm Solana..!")])
answer.content

"Hi Solana! It's nice to meet you. How are you doing today?"

In [4]:
answer02 = model.invoke([HumanMessage(content="What is my name?..!")])
answer02.content

"I'm sorry, but I cannot determine your name as I am an AI digital assistant and do not have access to personal information."

In [5]:
answer03 = model.invoke([
    HumanMessage(content='Hi~ I am Solana~'),
		AIMessage(content='Hello Bob! How can I assist you today? '),
    HumanMessage(content='Nothing!'),
	  ])

answer03.content

'Alright then! Have a great day, Solana!'

## [Message History]

In [6]:

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
	if session_id not in store:
		store[session_id] = InMemoryChatMessageHistory()	
    
	return store[session_id]

with_message_history = RunnableWithMessageHistory(
    model, 
    get_session_history
    )

config = {'configurable':{'session_id':'abc1'}}

resopnse = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Solana')],
	config=config,
)

resopnse.content

'Hello Solana! How are you doing today?'

In [7]:
resopnse = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
	config=config,
)

resopnse.content

'Your name is Solana.'

In [8]:
# 새로운 id 대화 생성
config = {"configurable":{"session_id":"1004"}}

resopnse = with_message_history.invoke(
    [HumanMessage(content='what is my name?')],
    config=config
    )

resopnse.content

"I'm sorry, I don't have access to that information."

---

## [Prompt templates] - MessagePlaceholder

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [('system','you are a helpful assistant. Answer all questions to the best of your ability.'),
    MessagesPlaceholder('messages'),]   #원래 이 곳엔 history가 와야함
)

chain = prompt | model
 
resopnse = chain.invoke({'messages':[HumanMessage(content='Hi~ I am Rachel')]})
resopnse.content

'Hello, Rachel! How can I assist you today?'

In [10]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history
    )

config = {'configurable':{'session_id':'abc5'}}

resopnse = with_message_history.invoke(
    [HumanMessage(content='Hello, I am Rachel.')],
	config=config
)

resopnse.content

'Hello Rachel! How can I assist you today?'

In [11]:
resopnse = with_message_history.invoke(
    [HumanMessage(content='What is my name???')],
	config=config
)

resopnse.content

'Your name is Rachel.'

In [12]:
for m in get_session_history('abc5').messages:
    print([m.type.upper()], m.content)

['HUMAN'] Hello, I am Rachel.
['AI'] Hello Rachel! How can I assist you today?
['HUMAN'] What is my name???
['AI'] Your name is Rachel.


프롬프트를 좀 더 복잡하게 만들어보자. 프롬프트 템플릿이 다음과 같다고 가정.

In [13]:
prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Answer in the language specified: {language}."),
        MessagesPlaceholder("messages"),
    ])

chain = prompt | model    #ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list })

# 이 복잡한 체인을 Message History 클래스로 래핑. (입력에 여러 키가 있으므로 채팅 기록을 저장 시 사용할 올바른 키를 지정해야함)

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,     #저장은 되지만 프롬프트에 넣지 못함 
    # history_messages_key='messages'   ===>원래 이렇게 인풋메시지키는 인풋으로 해줬음  
    input_messages_key='messages'
)

config = {'configurable':{'session_id':'abc11'}}

resopnse = with_message_history.invoke(
    {'messages':[HumanMessage(content='hi, i am Todd')], 'language':'Spanish'},
    config=config
    
)

resopnse.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [14]:
resopnse = with_message_history.invoke(
    {'messages':[HumanMessage(content='What is my name?')], 'language':'Japanese'},
    config=config
)
resopnse.content

'あなたの名前はトッドさんです。'

---

## [Managing Conversation History]

In [15]:
from langchain_core.messages import trim_messages, SystemMessage

trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",   
)

messages = [
    SystemMessage(content='You are a good psychological counselor.'),
    HumanMessage(content='Hi! I am Ruel'),
    AIMessage(content='Hi'),
    HumanMessage(content="I like chocolate ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good psychological counselor.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [16]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# cf) 이전의 chain : chain = prompt | model 

chain = ( 
    RunnablePassthrough.assign(messages=itemgetter('messages') | trimmer)             #===>assign: message에 itemgetter('message')|trimmer의 값을 할당해줌. 
	  | prompt
    | model 
    )

resopnse = chain.invoke({
    'messages': messages + [HumanMessage(content='내 이름이 뭐였니?')], 
    'language': 'Korean',
    })

resopnse
resopnse.content

'죄송합니다, 제가 이름을 알고있는 것은 아니지만, 제가 도울 수 있는 다른 질문이 있으면 언제든 물어봐 주세요!'

In [17]:
resopnse = chain.invoke(
    {
        "messages": messages + [HumanMessage(content='What math problem did i ask?')],
		"language": "Korean",
	}
)

resopnse.content

'미안하지만 질문을 기억하지 않아서 도와드리기 어렵습니다. 어떤 수학 문제에 대해 물어보셨나요? 부탁드립니다.'

In [18]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key='messages',
)

config = {'configurable':{'session_id':'abc20'}}

resopnse = with_message_history.invoke(
    {
    'messages': messages + [HumanMessage(content='What is my name?')],
    'language':'Korean',
    },
    config=config,
)

resopnse.content

"I'm sorry, but I am unable to determine your name as I do not have access to that information. How can I assist you further?"

In [19]:
for message in get_session_history('abc20').messages:
    print(f'[{message.type.upper()}]:{message.content}')

[SYSTEM]:You are a good psychological counselor.
[HUMAN]:Hi! I am Ruel
[AI]:Hi
[HUMAN]:I like chocolate ice cream
[AI]:nice
[HUMAN]:whats 2 + 2
[AI]:4
[HUMAN]:thanks
[AI]:no problem!
[HUMAN]:having fun?
[AI]:yes!
[HUMAN]:What is my name?
[AI]:I'm sorry, but I am unable to determine your name as I do not have access to that information. How can I assist you further?


In [20]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a specific math problem. How can I assist you with math today?"

In [22]:
config = {"configurable":{"session_id":"abc15"}}
for r in with_message_history.stream(
    {'messages':[HumanMessage(content='Hi I am Sol. Tell me a joke!')],
    'language':'English'},
    config=config
):
    print(r.content, end='|')

|Sure|,| Sol|!| Here|'s| a| joke| for| you|:| Why| did| the| scare|crow| win| an| award|?| Because| he| was| outstanding| in| his| field|!| 😄||